In [64]:
import requests
import json
import concurrent


API_KEY = '1783319260d70f84da21868ce0fd6207'

In [65]:
def write(response):
    with open('json_dir/lfg.json', 'w') as json_file:
        json_file.write(response.text)

In [66]:
def do_scrap(link):
    region = 'EU'
    #region = 'any'
    #link = f'https://api.tracker.gg/api/v1/valorant/lfg/search?region{region}&playlist=competitive&skill=any'
    #api_link = f"http://api.scraperapi.com?api_key={API_KEY}&url={link}"
    payload = {'api_key': API_KEY, 'url': link, 'keep_headers': 'true'}
    head = {
        'authority': 'api.tracker.gg',
        'method': 'GET',
        'path': '/api/v1/valorant/lfg/search?region=any&playlist=competitive&skill=any',
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'nb-NO,nb;q=0.9,no;q=0.8,en-US;q=0.7,en;q=0.6,ru;q=0.5',
        'cache-control': 'max-age=0',
        'cookie': '__cflb=02DiuFQAkRrzD1P1mdkJhfdTc9AmTWwYkL5dhc6BaWz7E; X-Mapping-Server=s14; __cf_bm=eCW52EohIqvzfugOyKvNvo4spvffbOwnBy5YZBvh4gQ-1653777883-0-AYz5PvgOpqpL7/UCjWN7/dokINYUw/EV1D4T6eY5n9YecMF2lY0M4TWWdDyySEalQ3lzxua3IYwD9KUjzZtmFdM=',
        'sec-ch-ua': 'Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': 'macOS',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36'
        }
    count = 0
    while True:
        response = requests.get('http://api.scraperapi.com', params=payload, headers=head)
        if response.status_code == 200:
            print(f'Success:',response.status_code)
            return response
        else:
            print(f'Failure:',response.status_code)
            count += ''
            if count >= 2:
                return None
            continue
#200	Successful response. - ScraperAPI
#400    Bad request is sent when the server cannot understand the request sent by the client. - Requests
#401    Unauthorized is sent whenever fulfilling the requests requires supplying valid credentials. - Requests
#403    Forbidden means that the server understood the request but will not fulfill it. - Requests
#404	Page requested does not exist. - ScraperAPI
#410	Page requested is no longer available. - ScraperAPI
#500	After retrying for 60 seconds, the API was unable to receive a successful response. - ScraperAPI
#429	You are sending requests too fast, and exceeding your concurrency limit. - ScraperAPI
#403	You have used up all your API credits. - ScraperAPI

In [81]:
def get_info():
    file = open('json_dir/lfg.json')
    data = json.load(file)['data']['entries']
    placement_ID = 0
    player_dictionary = {}
    for person in data:
        placement_ID += 1
        # Random valotrack account info
        userinfo = person['userInfo']
        
        userinfo_id = userinfo['userId']
        is_premium = userinfo['isPremium']
        is_verified = userinfo['isVerified']
        is_Influencer = userinfo['isInfluencer']
        region = userinfo['countryCode']
        is_suspicious = userinfo['isSuspicious']
        
        # Valorant info
        platform_info = person['platformInfo']
        
        valo_name = platform_info['platformUserHandle']
        avatar_url = platform_info['avatarUrl']
        
        # User bio/info
        bio = person['bio']
        
        has_mic = bio['hasMicrophone']
        playstyle = bio['playStyle']
        about = bio['about']
        
        # Valo account stats
        valo = person['profileSummary']
        
        rank_tier = bio['statsJson'].split(' ')[1].replace(',', '')
        rank = valo['featureStat']['value']
        winrate = valo['stats']['matchesWinPct']['value']
        kda = valo['stats']['kDRatio']['value']
        dmg_per_round = valo['stats']['damagePerRound']['value']

        # Get links to different medias
        social_accounts = userinfo['socialAccounts']
        socials = []
        for social in social_accounts:
            platform = social['platformSlug']
            user_handle = social['platformUserHandle']
            socials.append([platform, user_handle])
        taken_keys = player_dictionary.keys()
        if rank_tier in taken_keys:
            player_dictionary[rank_tier] = [player_dictionary[rank_tier],
                                            valo_name,
                                            rank, rank_tier,
                                            winrate, kda,
                                            dmg_per_round,
                                            region,
                                            socials,
                                            has_mic,
                                            playstyle,
                                            about,
                                            is_verified,
                                            is_premium,
                                            is_Influencer,
                                            is_suspicious,
                                            avatar_url,
                                            ]
        else:
            player_dictionary[rank_tier] = [valo_name,
                                            rank, rank_tier,
                                            winrate, kda,
                                            dmg_per_round,
                                            region,
                                            socials,
                                            has_mic,
                                            playstyle,
                                            about,
                                            is_verified,
                                            is_premium,
                                            is_Influencer,
                                            is_suspicious,
                                            avatar_url,
                                            ]        
    # dictionary_items = player_dictionary.items()
    # sorted_dictionary = sorted(dictionary_items) 
    # return sorted_dictionary
    return player_dictionary

In [68]:
region = 'EU'
    #region = 'any'
links = [f'https://api.tracker.gg/api/v1/valorant/lfg/search?region{region}&playlist=competitive&skill=any',
         f'https://api.tracker.gg/api/v1/valorant/lfg/search?region{region}&playlist=competitive&skill=any',
         f'https://api.tracker.gg/api/v1/valorant/lfg/search?region{region}&playlist=competitive&skill=any',
         f'https://api.tracker.gg/api/v1/valorant/lfg/search?region{region}&playlist=competitive&skill=any',
         f'https://api.tracker.gg/api/v1/valorant/lfg/search?region{region}&playlist=competitive&skill=any',
         ]


In [69]:
def all_in_one(links):
    write(do_scrap(links))
    get_info() 

In [82]:
response = get_info()
print(response)

{'9': [[[[[[[['marquez#1214', 'Silver 1', '9', 57.75, 1.27, 158.15, 'US', [['discord', 'marquez#7578']], False, None, 'NAE', False, True, False, None, 'https://trackercdn.com/cdn/user-avatars/1209128.gif?v=637888789816543134'], 'Cad#6969', 'Silver 1', '9', 46.25, 1.04, 144.97, 'GB', [['discord', 'Cad#9406']], False, None, 'LGF to clim. Ex plat', False, False, False, None, None], 'Shahmeer#8269', 'Silver 1', '9', 48.51, 0.9, 135.24, 'GB', [['discord', 'Shahmeer#2502']], False, None, '', False, False, False, None, None], 'Marrow#5208', 'Silver 1', '9', 49.09, 0.97, 131.07, 'DE', [['discord', 'Marrow#4192']], False, None, '', False, False, False, None, None], 'VetleJuice#4122', 'Silver 1', '9', 47.31, 0.99, 143.54, 'NO', [['discord', 'VetleJuice#5050']], False, None, 'Silver 1 peak silver 2 95rr', False, False, False, None, None], 'squaredash#6885', 'Silver 1', '9', 34.78, 1.48, 187.64, 'US', [['discord', 'square-#7639']], False, None, 'I want to learn and rank up.', False, False, False, 

In [71]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(all_in_one, links)


KeyboardInterrupt: 

Success: 200
